In [9]:
#import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import JobInput
from azure.ml._constants import AssetTypes
from azure.ml import dsl
from azure.identity import InteractiveBrowserCredential
import json

In [10]:
with open("./ml_config.json", "r") as read_file:
    config = json.load(read_file)

# Enter details of your AML workspace
subscription_id = config["workspace_settings"]["subscription_id"]
resource_group = config["workspace_settings"]["resource_group"]
workspace = config["workspace_settings"]["workspace"]

# Get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# Enter settings
compute_name = config["run_settings"]["compute_name"]
experiment_name = config["run_settings"]["experiment_name"]

# Pipeline inputs
data_input_uri = config["pipeline_inputs"]["input_data"]
feature_list = config["pipeline_inputs"]["feature_list"]
label_col = config["pipeline_inputs"]["label_col"]
learning_rate = config["pipeline_inputs"]["learning_rate"]
n_estimators = config["pipeline_inputs"]["n_estimators"]
test_size = config["pipeline_inputs"]["test_size"]
random_state = config["pipeline_inputs"]["random_state"]

# init compute on workspace
try:
    ml_client.compute.get(name=compute_name)
except Exception:
    print('Creating a new compute target...')
    from azure.ml.entities import AmlCompute
    compute = AmlCompute(
        name=compute_name,
        size="Standard_D2_v2",
        max_instances=2
    )
    ml_client.compute.begin_create_or_update(compute)

In [11]:
# Load pipeline definition from 
%run ./pipeline.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
pipeline = training_pipeline_func(
    input_data=JobInput(type=AssetTypes.URI_FILE, path=data_input_uri),
    feature_list=feature_list,
    label_col=label_col,
    learning_rate=learning_rate,
    n_estimators=n_estimators
)

pipeline.settings.default_compute = config["run_settings"]["compute_name"]
# print(pipeline)

In [8]:
# submit pipeline to cloud
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name=experiment_name)
print(f'Job link: {pipeline_job.services["Studio"].endpoint}')

Uploading components (0.02 MBs): 100%|##########| 16258/16258 [00:01<00:00, 13301.98it/s]




Job link: https://ml.azure.com/runs/amusing_melon_43ywdfjpj2?wsid=/subscriptions/ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9/resourcegroups/pipeline-pm/workspaces/prod&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
